# Using the `qbraid-SDK`
qBraid SDK is our flagship Python toolkit to run platform agnostic quantum programs. We have built the `qbraid.runtime` module to interface with multiple quantum software packages and their respective vendors. 

In [1]:
import qbraid
qbraid.__version__

'0.9.7'

## Using the `QbraidProvider` and Devices
- We use the provider abstraction to associate together all the devices available via qBraid
- Each provider has a list of supported devices which can be retrieved via the `.get_devices()` method

In [2]:
from qbraid.runtime import QbraidProvider
provider = QbraidProvider()

In [3]:
provider.get_devices()

[<qbraid.runtime.native.device.QbraidDevice('aws_sv1')>,
 <qbraid.runtime.native.device.QbraidDevice('aws_tn1')>,
 <qbraid.runtime.native.device.QbraidDevice('quera_aquila')>,
 <qbraid.runtime.native.device.QbraidDevice('aws_dm1')>,
 <qbraid.runtime.native.device.QbraidDevice('qbraid_qir_simulator')>,
 <qbraid.runtime.native.device.QbraidDevice('iqm_garnet')>,
 <qbraid.runtime.native.device.QbraidDevice('quera_qasm_simulator')>,
 <qbraid.runtime.native.device.QbraidDevice('ionq_simulator')>,
 <qbraid.runtime.native.device.QbraidDevice('nec_vector_annealer')>,
 <qbraid.runtime.native.device.QbraidDevice('ionq_aria_1')>,
 <qbraid.runtime.native.device.QbraidDevice('rigetti_ankaa_3')>]

### Choose AWS Statevector Simulator

In [4]:
aws_sim = provider.get_device("aws_sv1") 
aws_sim

<qbraid.runtime.native.device.QbraidDevice('aws_sv1')>

In [5]:
aws_sim._target_spec

<ProgramSpec('braket.circuits.circuit.Circuit', 'braket')>

## Define the quantum circuit

We will build the following circuit representing [the **Bell** state](https://en.wikipedia.org/wiki/Bell_state) - 

$$ | \Psi \rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} $$

##  Submit Cirq circuit to AWS simulator

### Build a Cirq circuit
Cirq is Google's quantum computing library

In [7]:
import cirq

# Create 2 qubits
qubits = [cirq.LineQubit(i) for i in range(2)]

# Define the circuit
bell_circuit = cirq.Circuit()

# Apply a Hadamard gate to the first qubit
bell_circuit.append(cirq.H(qubits[0]))

# Apply CNOT gates to entangle the qubits
bell_circuit.append(cirq.CNOT(qubits[0], qubits[1]))

# Measure all qubits
bell_circuit.append(cirq.measure(*qubits, key='result'))

In [8]:
print(bell_circuit)

0: ───H───@───M('result')───
          │   │
1: ───────X───M─────────────


### Execute cirq on AWS simulator

In [9]:
job = aws_sim.run(bell_circuit, shots = 1000) 
print(job.result().data.measurement_counts) 

{'11': 522, '00': 478}


##  Submit qiskit circuit to AWS simulator 

### Build a Qiskit circuit
Qiskit is IBM's quantum programming library

In [10]:
from qiskit import QuantumCircuit   

# Create a 2-qubit quantum circuit with 2 classical bits for measurement
qiskit_qc = QuantumCircuit(2)

qiskit_qc.h(0)
qiskit_qc.cx(0,1)

qiskit_qc.measure_all()

In [11]:
print(qiskit_qc)

        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 


### Execute qiskit on AWS simulator

In [12]:
job = aws_sim.run(qiskit_qc, shots = 1000)
print(job.result().data.measurement_counts) 

{'11': 478, '00': 522}


## Running on real hardware : Rigetti Ankaa - 3

In [13]:
ankaa_device_id = "rigetti_ankaa_3"
ankaa_device = provider.get_device(ankaa_device_id) 
ankaa_device

<qbraid.runtime.native.device.QbraidDevice('rigetti_ankaa_3')>

In [23]:
ankaa_device.metadata()

{'device_id': 'rigetti_ankaa_3',
 'simulator': False,
 'num_qubits': 82,
 'provider_name': 'Rigetti',
 'basis_gates': None,
 'name': 'Ankaa-3 (AWS)',
 'pricing': {'perTask': Credits('30.0'),
  'perShot': Credits('0.0900'),
  'perMinute': Credits('0')},
 'queue_depth': 0,
 'status': 'ONLINE',
 'paradigm': 'gate_model',
 'runtime_config': {'target_program_type': 'braket',
  'conversion_scheme': {'conversion_graph': <qbraid.transpiler.graph.ConversionGraph at 0x7c72d630e8f0>,
   'max_path_attempts': 3,
   'max_path_depth': None},
  'options': {'transpile': True,
   'transform': True,
   'validate': 2,
   'prepare': True}}}

We will again utilise the **Qiskit circuit** that we developed in the above block -

In [14]:
num_shots = 50 # Review device pricing to calculate the cost of the experiment. Do not increase to a very high value. 
job = ankaa_device.run(qiskit_qc, shots= num_shots) 
print(job) 

<QbraidJob(id:'rigetti_ankaa_3-harshit-qjob-723iocdix04zo79cb1oe')>


In [15]:
job.status()

<INITIALIZING: 'job is being initialized'>

In [16]:
print("Final result: ", job.result())

Final result:  Result(
  device_id=rigetti_ankaa_3,
  job_id=rigetti_ankaa_3-harshit-qjob-723iocdix04zo79cb1oe,
  success=True,
  data=GateModelResultData(measurement_counts={'11': 18, '00': 31, '01': 1}, measurements=array(shape=(0,), dtype=object)),
  status='COMPLETED',
  statusText='job has successfully run',
  shots=50,
  experimentType=GATE_MODEL,
  queuePosition=None,
  timeStamps={createdAt: 2025-07-07T06:06:46Z, endedAt: 2025-07-07T06:06:48Z, executionDuration: None},
  tags={},
  cost=Credits('34.5'),
  preflight=False,
  schemaHeader={name: 'qbraid.runtime.schemas.job', version: 1.0},
  metadata={openQasm: '...', circuitNumQubits: 2, circuitDepth: 3}
)


In [19]:
print("Final counts : ", job.result().data.measurement_counts)

Final counts :  {'11': 18, '00': 31, '01': 1}


- We can clearly see that the majority counts are from the $|00\rangle$ and the $|11\rangle$ state
- Since this is a **real quantum device**, we are bound to get some **noisy results**. This is highlighted from the fact that 
we get some shots from other possible states i.e. $|01\rangle$ and $|10\rangle$ 